<a href="https://colab.research.google.com/github/buabaj/Mobile-data-outage-prediction/blob/main/CD2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
data = pd.read_excel('SD2.xlsx')

In [49]:
data.isnull().sum()

REGION      0
CAUSE       0
CATEGORY    0
MTTR        0
dtype: int64

In [50]:
!pip install xplore
from xplore.data import xplore

In [51]:
xplore(data)

------------------------------------
The fist 5 entries of your dataset are:

   REGION  CAUSE  CATEGORY         MTTR
0       8      6         1    75.000000
1       4      6         1  2306.983333
2       8      6         1   292.000000
3       4      8         1   265.733333
4       5      4         0    33.000000


------------------------------------
The last 5 entries of your dataset are:

      REGION  CAUSE  CATEGORY        MTTR
2055       4      7         0    9.483333
2056     124      8         0    8.850000
2057       9      2         0  147.383333
2058       3      9         0    3.850000
2059       8      6         0   68.900000


------------------------------------
Stats on your dataset:

<bound method NDFrame.describe of       REGION  CAUSE  CATEGORY         MTTR
0          8      6         1    75.000000
1          4      6         1  2306.983333
2          8      6         1   292.000000
3          4      8         1   265.733333
4          5      4         0    33.00

In [52]:
df = pd.DataFrame(data)
df

,REGION,CAUSE,CATEGORY,MTTR
0,8,6,1,75.000000
1,4,6,1,2306.983333
2,8,6,1,292.000000
3,4,8,1,265.733333
4,5,4,0,33.000000
...,...,...,...,...
2055,4,7,0,9.483333
2056,124,8,0,8.850000
2057,9,2,0,147.383333
2058,3,9,0,3.850000


In [53]:
## enter the feature columns you want to assing to the dependent and independent variables here

X = df[['CAUSE']]
y = df[['CATEGORY']]

In [54]:
seed = 50  # so that the result is reproducible
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state = seed)

In [55]:
X_train = X_train.fillna('na')
X_test = X_test.fillna('na')

In [56]:
X_train.dtypes

CAUSE    int64
dtype: object

In [57]:
features_to_encode = list(X_train.select_dtypes(include = ['object']).columns)

In [58]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
col_trans = make_column_transformer(
                        (OneHotEncoder(),features_to_encode),
                        remainder = "passthrough"
                        )

In [59]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(
                      min_samples_leaf=50,
                      n_estimators=150,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=seed,
                      max_features='auto')

In [60]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(col_trans, rf_classifier)
np.array(X_train, dtype=np.float32)
pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  [])],
                                   verbose=False)),
                ('randomforestclassifier',
                 RandomForestClas...ap=True, ccp_alpha=0.0,
                                        class_weight

In [61]:
y_pred = pipe.predict(X_test)

In [62]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score

In [63]:
accuracy_score(y_test, y_pred)
print(f"The accuracy of the model is {round(accuracy_score(y_test,y_pred),3)*100} %")

The accuracy of the model is 74.3 %


In [64]:
train_probs = pipe.predict_proba(X_train)[:,1] 
probs = pipe.predict_proba(X_test)[:, 1]
train_predictions = pipe.predict(X_train)

In [65]:
print(f'Train ROC AUC Score: {roc_auc_score(y_train, train_probs)}')
print(f'Test ROC AUC  Score: {roc_auc_score(y_test, probs)}')

Train ROC AUC Score: 0.5760600294402758
Test ROC AUC  Score: 0.6349889012208657


In [66]:
def evaluate_model(y_pred, probs,train_predictions, train_probs):

    baseline = {}    
    baseline['recall']=recall_score(y_test, [1 for _ in range(len(y_test))])    
    baseline['precision'] = precision_score(y_test,[1 for _ in range(len(y_test))])    
    baseline['roc'] = 0.5

    for metric in ['recall', 'precision', 'roc']:  
      print(f'{metric.capitalize()}')

    results = {}
    results['recall'] = recall_score(y_test, y_pred)
    results['precision'] = precision_score(y_test, y_pred)
    results['roc'] = roc_auc_score(y_test, probs)

    train_results = {}
    train_results['recall'] = recall_score(y_train,       train_predictions)
    train_results['precision'] = precision_score(y_train, train_predictions)
    train_results['roc'] = roc_auc_score(y_train, train_probs)

    Baseline = {round(baseline[metric], 2)}

    Test = {round(results[metric], 2)}

    Train = {round(train_results[metric], 2)}
    
    labels=np.unique(y_pred)
    return(Baseline, Test, Train)

In [67]:
# Calculate false positive rates and true positive rates and plot ROC curves
base_fpr, base_tpr, _ = roc_curve(y_test, [1 for _ in range(len(y_test))])
model_fpr, model_tpr, _ = roc_curve(y_test, probs)
plt.figure(figsize = (8, 6))

plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
plt.plot(model_fpr, model_tpr, 'r', label = 'model')

plt.legend();
plt.xlabel('False Positive Rate');
plt.ylabel('True Positive Rate'); plt.title('ROC Curves');
plt.savefig('roc_curves.jpg');


In [68]:
evaluate_model(y_pred,probs,train_predictions,train_probs)

Recall
Precision
Roc


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


({0.5}, {0.63}, {0.58})

In [69]:
print(rf_classifier.feature_importances_)
print(f" There are {len(rf_classifier.feature_importances_)} features in total")

[1.]
 There are 1 features in total
